10) A partir de la plataforma online de los paises en los que opera, Nintendo tiene info de ventas de videojuegos diarias digitales por pais en el siguiente RDD: (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias). Por otro lado se tiene otro RDD con info de todos los videojuegos que se venden con formato (id_videojuego, titulo, rating_pegi, rating_esbr). Tener en cuenta que el mismo videojuego se puede vender en distintos paises y esos nos permitira obtener metricas a nivel global. **Con esta info escribir un programa en PySpark que permita:
-A) Obtener el videojuego con mas ventas digitales globales en un RDD con formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales;
-B) Para el videojuego con mas ventas, obtener cual es el pais para el cual se registra una mayor tasa de conversion (es decir, mayor total_ventas_diarias/visitas_diarias)**

In [4]:
import random
from fechasRandom import *

ventasJuegos = []
infoJuegos = []
titulos = ['Kirby','Super Mario Maker 2',"Luigi's Mansion",'Mario Kart 8 Deluxe','The Legend of Zelda','Super Mario Bros','Astral Chain']

fechaInicial = '2019-01-01'
fechaFinal = '2020-01-01'

for _ in range(50):
    idVideojuego = random.randint(1,7)
    codigoPais = random.randint(1,10)
    fecha = random_date(fechaInicial, fechaFinal)
    visitasDiarias = random.randint(1,500)
    totalVentasDiarias = random.randint(1,350)
    ventasJuegos.append((idVideojuego,codigoPais,fecha,visitasDiarias,totalVentasDiarias))
    
    titulo = titulos[idVideojuego-1]
    ratingPegi = random.randint(1,100)
    ratingEsbr = random.randint(1,100)
    infoJuegos.append((idVideojuego,titulo,ratingPegi,ratingEsbr))
    
ventasJuegosRDD = sc.parallelize(ventasJuegos)
infoJuegosRDD = sc.parallelize(infoJuegos)

In [5]:
ventasJuegosRDD.collect()

[(4, 3, '2019-08-19', 435, 148),
 (3, 1, '2019-12-12', 313, 88),
 (1, 9, '2019-09-19', 394, 322),
 (1, 9, '2019-07-31', 181, 201),
 (6, 6, '2019-10-13', 353, 194),
 (5, 8, '2019-09-21', 131, 155),
 (2, 2, '2019-03-15', 390, 9),
 (6, 8, '2019-12-11', 146, 11),
 (4, 1, '2019-08-12', 178, 151),
 (2, 1, '2019-01-22', 349, 345),
 (4, 5, '2019-05-27', 291, 274),
 (3, 3, '2019-08-05', 261, 262),
 (7, 3, '2019-04-26', 454, 82),
 (4, 1, '2019-08-24', 84, 94),
 (4, 9, '2019-06-05', 160, 84),
 (5, 4, '2019-08-16', 252, 24),
 (7, 3, '2019-09-16', 30, 149),
 (6, 10, '2019-11-16', 441, 309),
 (3, 6, '2019-11-17', 104, 260),
 (7, 3, '2019-04-30', 371, 343),
 (4, 3, '2019-04-13', 33, 29),
 (2, 6, '2019-04-11', 67, 167),
 (6, 5, '2019-08-21', 246, 308),
 (3, 5, '2019-07-23', 264, 98),
 (1, 1, '2019-01-28', 173, 337),
 (2, 3, '2019-10-20', 55, 284),
 (4, 2, '2019-10-05', 22, 10),
 (7, 9, '2019-01-02', 59, 157),
 (3, 9, '2019-07-08', 136, 299),
 (1, 9, '2019-03-25', 408, 303),
 (1, 6, '2019-04-14', 195, 

In [6]:
infoJuegosRDD.collect()

[(4, 'Mario Kart 8 Deluxe', 66, 21),
 (3, "Luigi's Mansion", 53, 94),
 (1, 'Kirby', 26, 84),
 (1, 'Kirby', 16, 6),
 (6, 'Super Mario Bros', 56, 19),
 (5, 'The Legend of Zelda', 97, 25),
 (2, 'Super Mario Maker 2', 51, 88),
 (6, 'Super Mario Bros', 100, 13),
 (4, 'Mario Kart 8 Deluxe', 59, 12),
 (2, 'Super Mario Maker 2', 56, 89),
 (4, 'Mario Kart 8 Deluxe', 33, 41),
 (3, "Luigi's Mansion", 24, 77),
 (7, 'Astral Chain', 27, 62),
 (4, 'Mario Kart 8 Deluxe', 84, 44),
 (4, 'Mario Kart 8 Deluxe', 74, 1),
 (5, 'The Legend of Zelda', 71, 100),
 (7, 'Astral Chain', 95, 56),
 (6, 'Super Mario Bros', 16, 84),
 (3, "Luigi's Mansion", 54, 91),
 (7, 'Astral Chain', 34, 11),
 (4, 'Mario Kart 8 Deluxe', 76, 15),
 (2, 'Super Mario Maker 2', 9, 91),
 (6, 'Super Mario Bros', 59, 1),
 (3, "Luigi's Mansion", 56, 73),
 (1, 'Kirby', 41, 13),
 (2, 'Super Mario Maker 2', 70, 33),
 (4, 'Mario Kart 8 Deluxe', 10, 73),
 (7, 'Astral Chain', 77, 32),
 (3, "Luigi's Mansion", 4, 22),
 (1, 'Kirby', 63, 36),
 (1, 'Kir

**-A) Obtener el videojuego con mas ventas digitales globales en un RDD con formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales**

In [45]:
#(idJuego, totalVentasDiarias)
juegoCantidadVentas = ventasJuegosRDD.map(lambda x: (x[0],x[4]))
#(idJuego, totalVentasGlobal)
juegoCantidadVentas = juegoCantidadVentas.reduceByKey(lambda x,y: x+y)
juegoCantidadVentas = juegoCantidadVentas.takeOrdered(juegoCantidadVentas.count(), lambda x: -x[1])
juegoMasVendido = juegoCantidadVentas[0]

juegoTopVentasRDD = infoJuegosRDD.map(lambda x: (x[0],x[1]))
juegoTopVentasRDD = juegoTopVentasRDD.filter(lambda x: x[0] == juegoMasVendido[0])
juegoTopVentasRDD = sc.parallelize(juegoTopVentasRDD.take(1))
juegoTopVentasRDD.map(lambda x: (x[0],x[1],juegoMasVendido[1])).collect()

[(6, 'Super Mario Bros', 1846)]

**-B) Para el videojuego con mas ventas, obtener cual es el pais para el cual se registra una mayor tasa de conversion (es decir, mayor total_ventas_diarias/visitas_diarias)**

In [50]:
mayorTasaConversion = ventasJuegosRDD.filter(lambda x: x[0] == juegoMasVendido[0])
mayorTasaConversion = mayorTasaConversion.map(lambda x: (x[1],x[4]/x[3]))
mayorTasaConversion = mayorTasaConversion.takeOrdered(mayorTasaConversion.count(),lambda x: -x[1])
mayorTasaConversion[0][0]

10